In [1]:
import pandas as pd

raw=pd.read_csv("./raw/H28.csv", index_col=0)
raw

,第1層（大項目）,第2層（中項目）,第3層（小項目）,第4層（細小項目）,第5層（学修目標）
index,,,,,
1,A 医師として求められる基本的な資質・能力,A-1 プロフェッショナリズム,A-1-1) 医の倫理と生命倫理,なし,①医学・医療の歴史的な流れとその意味を概説できる。
2,A 医師として求められる基本的な資質・能力,A-1 プロフェッショナリズム,A-1-1) 医の倫理と生命倫理,なし,②臨床倫理や生と死に関わる倫理的問題を概説できる。
3,A 医師として求められる基本的な資質・能力,A-1 プロフェッショナリズム,A-1-1) 医の倫理と生命倫理,なし,③ヒポクラテスの誓い、ジュネーブ宣言、医師の職業倫理指針、医師憲章等医療の倫理に関する規範を...
4,A 医師として求められる基本的な資質・能力,A-1 プロフェッショナリズム,A-1-2) 患者中心の視点,なし,①リスボン宣言等に示された患者の基本的権利を説明できる。
5,A 医師として求められる基本的な資質・能力,A-1 プロフェッショナリズム,A-1-2) 患者中心の視点,なし,②患者の自己決定権の意義を説明できる。
...,...,...,...,...,...
2002,G 臨床実習,G-4 診療科臨床実習,G-4-4) シミュレーション教育,なし,①シミュレータを用いて反復練習をすることで、臨床技能を磨く。
2003,G 臨床実習,G-4 診療科臨床実習,G-4-4) シミュレーション教育,なし,②模擬患者の協力を得て、臨床技能（コミュニケーションスキルを含む）や医療者に求められる態度を...
2004,G 臨床実習,G-4 診療科臨床実習,G-4-4) シミュレーション教育,なし,③シナリオを用いたトレーニングを通して、状況判断、意思決定能力を獲得する。


In [2]:
import re

data=pd.DataFrame([])
data["id1"]=raw["第1層（大項目）"].str.extract(r"^(.)")
data["text1"]=raw["第1層（大項目）"].str.extract(r"^. ?(.+)")
data["id2"]=raw["第2層（中項目）"].str.extract(r"^.\-(\d+)")
data["id2"]=data["id1"]+"-"+data["id2"].str.zfill(2)
data["text2"]=raw["第2層（中項目）"].str.extract(r"^.\-\d+ (.+)")
data["id3"]=raw["第3層（小項目）"].str.extract(r"^.\-\d+\-(\d+)")
data["id3"]=data["id2"]+"-"+data["id3"].str.zfill(2)
data["text3"]=raw["第3層（小項目）"].str.extract(r"^.\-\d+\-\d+\) (.+)")
raw["id3"]=data["id3"]

id4_list=[]
text4_list=[]
current_parent=""
prev_text=""
current_index=0
for index,row in raw.iterrows():
  text=row["第4層（細小項目）"]
  parent=row["id3"]
  if parent!= current_parent:
    current_index=0
    prev_text=""
  if prev_text!= text:
    current_index=current_index+1
  current_parent=parent
  prev_text=text
  if text=="なし":
    id4_list.append(f"{parent}-na")
    text4_list.append(text)
  else:
    id4_list.append(f"{parent}-{str(current_index).zfill(2)}")
    text4_list.append(re.sub(r"^.\-\d+\-\d+\)\-\(\d+\) ","",str(text)))

data["id4"]=id4_list
data["text4"]=text4_list
raw["id4"]=data["id4"]

id5_list=[]
text5_list=[]
current_parent=""
prev_text=""
current_index=0
for index,row in raw.iterrows():
  text=row["第5層（学修目標）"]
  parent=row["id4"]
  if parent!= current_parent:
    current_index=0
    prev_text=""
  if prev_text!= text:
    current_index=current_index+1
  current_parent=parent
  prev_text=text
  if text=="なし":
    id5_list.append(f"{parent}-na")
    text5_list.append(text)
  else:
    id5_list.append(f"{parent}-{str(current_index).zfill(2)}")
    item_text=re.sub(r"^([.０-９0-9]{1,2})( |\.|．)","",str(text))
    item_text=re.sub(r"^[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖]","",str(item_text))
    text5_list.append(item_text)

data["id5"]=id5_list
data["text5"]=text5_list


data.to_csv("./dist/H28.csv",encoding="utf_8_sig")
data

,id1,text1,id2,text2,id3,text3,id4,text4,id5,text5
index,,,,,,,,,,
1,A,医師として求められる基本的な資質・能力,A-01,プロフェッショナリズム,A-01-01,医の倫理と生命倫理,A-01-01-na,なし,A-01-01-na-01,医学・医療の歴史的な流れとその意味を概説できる。
2,A,医師として求められる基本的な資質・能力,A-01,プロフェッショナリズム,A-01-01,医の倫理と生命倫理,A-01-01-na,なし,A-01-01-na-02,臨床倫理や生と死に関わる倫理的問題を概説できる。
3,A,医師として求められる基本的な資質・能力,A-01,プロフェッショナリズム,A-01-01,医の倫理と生命倫理,A-01-01-na,なし,A-01-01-na-03,ヒポクラテスの誓い、ジュネーブ宣言、医師の職業倫理指針、医師憲章等医療の倫理に関する規範を概...
4,A,医師として求められる基本的な資質・能力,A-01,プロフェッショナリズム,A-01-02,患者中心の視点,A-01-02-na,なし,A-01-02-na-01,リスボン宣言等に示された患者の基本的権利を説明できる。
5,A,医師として求められる基本的な資質・能力,A-01,プロフェッショナリズム,A-01-02,患者中心の視点,A-01-02-na,なし,A-01-02-na-02,患者の自己決定権の意義を説明できる。
...,...,...,...,...,...,...,...,...,...,...
2002,G,臨床実習,G-04,診療科臨床実習,G-04-04,シミュレーション教育,G-04-04-na,なし,G-04-04-na-01,シミュレータを用いて反復練習をすることで、臨床技能を磨く。
2003,G,臨床実習,G-04,診療科臨床実習,G-04-04,シミュレーション教育,G-04-04-na,なし,G-04-04-na-02,模擬患者の協力を得て、臨床技能（コミュニケーションスキルを含む）や医療者に求められる態度を身...
2004,G,臨床実習,G-04,診療科臨床実習,G-04-04,シミュレーション教育,G-04-04-na,なし,G-04-04-na-03,シナリオを用いたトレーニングを通して、状況判断、意思決定能力を獲得する。
